In [1]:
import funzioni02 as f2
from copy import deepcopy
import numpy as np
import datetime
import qiskit.ignis.mitigation.measurement as mc
from qiskit import Aer, assemble, QuantumCircuit, QuantumRegister, ClassicalRegister, IBMQ, transpile, execute
from qiskit.providers.aer import AerSimulator, QasmSimulator
from qiskit.opflow import Zero, One, I, X, Y, Z
from qiskit.ignis.verification.tomography import state_tomography_circuits, StateTomographyFitter
from qiskit.quantum_info import state_fidelity
import matplotlib.pyplot as plt

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-community', group='ibmquantumawards', project='open-science-22')

sim_noisy_jakarta = QasmSimulator.from_backend(provider.get_backend('ibmq_jakarta'))
backend_sim_jakarta = sim_noisy_jakarta 
backend = provider.get_backend('ibmq_jakarta')
backend_sim = Aer.get_backend('qasm_simulator')

# Defining all the quantities

In [2]:
shots=8192
reps=2        # al massimo 10
precision=40
time=np.pi
backend_aus=backend_sim_jakarta
check=["4copy_check", [0,2,4,6]]
evo_type="column_evolution" # "column_evolution" or "complete_evolution" or "trotter_steps"
measure_mitigation = "yes"  #"yes" or "no"
steps=42

# Defining the evolution circuits

In [3]:
qcs_reps = []

for _ in range(reps):
    check=["4copy_check", [0,2,4,6]]
    qcs, qcs_na = f2.evolution_tomo(type=evo_type, N_steps=steps, tempo=time, precision=precision, initial_state='110', check=check)
    qcs_reps=qcs_reps + qcs
    
    check=["4copy_checkDD", [0,2,4,6]]
    qcs, qcs_na = f2.evolution_tomo(type=evo_type, N_steps=steps, tempo=time, precision=precision, initial_state='110', check=check)
    qcs_reps=qcs_reps + qcs

reps=reps*2

# Defining the calibration cirquits

In [4]:
check=["4copy_check", [0,2,4,6]]
qcs_calibs, meas_calibs = f2.calibration_cirquits(evo_type+"_remake", q_anc=check[1], check="yes", check_type=check[0])
state_lables = f2.bin_list(3+len(check[1]))

check=["4copy_checkDD", [0,2,4,6]]
qcs_calibsY, meas_calibsY = f2.calibration_cirquits(evo_type+"_remake", q_anc=check[1], check="yes", check_type=check[0])
state_lablesY = f2.bin_list(3+len(check[1])) 

# Running all the cirquits

In [5]:
#qcs_calibs[-1].draw(output="mpl")

In [6]:
job_evo=execute(qcs_reps, backend=backend_aus, shots=shots)

job_cal=execute(meas_calibs, backend=backend_aus, shots=shots)
job_cal_our=execute(qcs_calibs, backend=backend_aus, shots=shots)

job_calY=execute(meas_calibsY, backend=backend_aus, shots=shots)
job_cal_ourY=execute(qcs_calibsY, backend=backend_aus, shots=shots)

jobs_evo_res=[]


In [7]:

for i in range(reps):
    
    job=execute(qcs,backend=backend_sim, shots=10)
    results = job.result()
    
    for j in range(27):
        results.results[j].data.counts = job_evo.result().get_counts()[i*27+j]
    
    #print(results.get_counts()[0])
    jobs_evo_res.append(results)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


# Making the mitigation and computing the fidelity

In [8]:
meas_fitter = mc.CompleteMeasFitter(job_cal.result(), state_labels=state_lables)
meas_fitter_our = mc.CompleteMeasFitter(job_cal_our.result(), state_labels=state_lables)
meas_fitterY = mc.CompleteMeasFitter(job_calY.result(), state_labels=state_lablesY)
meas_fitter_ourY = mc.CompleteMeasFitter(job_cal_ourY.result(), state_labels=state_lablesY)

target_state = (One^One^Zero).to_matrix()

fids=np.zeros([reps,4])
fids_mean=np.zeros(4)
fids_dev=np.zeros(4)

fidsY=np.zeros([reps,4])
fids_meanY=np.zeros(4)
fids_devY=np.zeros(4)

i=0
iy=0
for j in range(len(jobs_evo_res)):
    print(j)
    if j%2==0:
        
        res=deepcopy(jobs_evo_res[i])

        new_res, new_res_nm = f2.mitigate(res, Measure_Mitig="yes", ancillas_conditions=['0011','1110','1101'], meas_fitter=meas_fitter)
        new_res_our, new_res_nm = f2.mitigate(res, Measure_Mitig="yes", ancillas_conditions=['0011','1110','1101'], meas_fitter=meas_fitter_our)
        new_res_not_mitigated = f2.mitigate(res, Measure_Mitig="no", ancillas_conditions=f2.bin_list(4))

        fids[i,0] = f2.fidelity_count(new_res_not_mitigated, qcs_na, target_state)
        fids[i,1] = f2.fidelity_count(new_res_nm, qcs_na, target_state)
        fids[i,2] = f2.fidelity_count(new_res, qcs_na, target_state)
        fids[i,3] = f2.fidelity_count(new_res_our, qcs_na, target_state)

        i+=1
    else:
        res=deepcopy(jobs_evo_res[i])

        new_resY, new_res_nmY = f2.mitigate(res, Measure_Mitig="yes", ancillas_conditions=['0011','1110','1101'], meas_fitter=meas_fitterY)
        new_res_ourY, new_res_nmY = f2.mitigate(res, Measure_Mitig="yes", ancillas_conditions=['0011','1110','1101'], meas_fitter=meas_fitter_ourY)
        new_res_not_mitigatedY = f2.mitigate(res, Measure_Mitig="no", ancillas_conditions=f2.bin_list(4))

        fidsY[iy,0] = f2.fidelity_count(new_res_not_mitigatedY, qcs_na, target_state)
        fidsY[iy,1] = f2.fidelity_count(new_res_nmY, qcs_na, target_state)
        fidsY[iy,2] = f2.fidelity_count(new_resY, qcs_na, target_state)
        fidsY[iy,3] = f2.fidelity_count(new_res_ourY, qcs_na, target_state)

        iy+=1

for i in range(4):
    fids_mean[i]=np.mean(fids[:,i])
    fids_dev[i]=np.std(fids[:,i])

    fids_meanY[i]=np.mean(fidsY[:,i])
    fids_devY[i]=np.std(fidsY[:,i])

0
1
2
3


In [9]:
print(new_res_nm.get_counts(3).values())

dict_values([780, 899, 851, 820, 843, 907, 895, 789])


In [10]:
new_res_ourY.get_counts(-1)

{'000': 5.69384669185736e-11,
 '001': 5.458670989076464e-11,
 '010': 337.50674918661946,
 '011': 2.1221266197131067e-10,
 '100': 23.255030694936305,
 '101': 3.084630386262454e-10,
 '110': 7629.767967690524,
 '111': 13.270894032921298}

In [11]:
for i in range(4):
    print("no: ", fids_mean[i], "+-" ,fids_dev[i])
    print("yes: ",fids_meanY[i], "+-" ,fids_devY[i])
'''
mean_DD=0
mean_=0
for i in range(reps):
    if i%2==0:
        mean_+=fids[i,3]
    if i%2==1:
        mean_DD+=fids[i,3]
mean_DD=mean_DD/reps*2
mean_=mean_/reps*2

print(mean_)
print(mean_DD)'''

no:  0.40438313407910276 +- 0.40438364645362573
yes:  0.4043176166393265 +- 0.4043179921924885
no:  0.4249902949120232 +- 0.4249904763071882
yes:  0.42532839322460925 +- 0.42532928465183534
no:  0.4671808820171604 +- 0.46718092807203987
yes:  0.46801276039279305 +- 0.46801430018385354
no:  0.4778896657808589 +- 0.47789075759331456
yes:  0.47783160311615325 +- 0.477831714882883


'\nmean_DD=0\nmean_=0\nfor i in range(reps):\n    if i%2==0:\n        mean_+=fids[i,3]\n    if i%2==1:\n        mean_DD+=fids[i,3]\nmean_DD=mean_DD/reps*2\nmean_=mean_/reps*2\n\nprint(mean_)\nprint(mean_DD)'

In [12]:
fids

array([[0.80967664, 0.8505359 , 0.9340684 , 0.95433466],
       [0.80785589, 0.84942528, 0.93465513, 0.957224  ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ]])

In [13]:
fidsY

array([[0.80785589, 0.84942528, 0.93432771, 0.95520101],
       [0.80941457, 0.85188829, 0.93772333, 0.9561254 ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ]])